# MODELOS PANEL Y SUS APLICACIONES EN R

Los modelos panel normalmente se utilizan cuando el fenómeno económico, financiero, social, etc. que se está analizando tiene un componente de desagregación, corte trasversal o sección cruzada  y otro de series de tiempo 

 Los modelos panel clásicos más usados son los de efecto común, efecto individual fijo y el de efecto individual aleatorio.  Para sus aplicaciones es importante elaborar pruebas de hipótesis entre los supuestos de efectos comunes versus individuales, y entre los efectos individuales  fijos o aleatorios. Así los supuestos econométricos básicos de varianza homoscedastica, normalidad, no-autocorrelación serial y/o contemporánea.

##### Datos

La base de datos fue construida por Mendoza, M.A. (2013) y  contiene la inflación (INF), medida por la tasa de crecimiento del índice nacional de precios al consumidor por ciudad, la tasa de desocupación (U) por ciudad,   la tasa de desocupación natural (UN) por ciudad, estimada con el filtro Hodrick-Prescot, y la tasa de interés medida con CETES a 28 días (CETES28). 
 
El índice nacional de precios al consumidor y los CETES28 se construyeron con base al Banco de México (BANXICO) y la tasa de desocupación al INEGI.

##### Análisis Exploratorio

Con el objetivo de llevar a cabo el análisis exploratorio, desde la consola de R, se escribió el siguiente comando para tener activo la base de datos construida y guardada previamente en el formato de R (RData). 

In [1]:
load("InflacionDesempleoCiudades.RData")

In [2]:
attach(BDatos)
summary(BDatos)

     Ciudad         Periodo              Nom_Ciudad       INF       
 Min.   : 1.00   Min.   :1995   LEON          : 17   Min.   : 2.30  
 1st Qu.: 7.75   1st Qu.:1999   ACAPULCO      : 16   1st Qu.: 4.40  
 Median :14.50   Median :2002   AGUASCALIENTES: 16   Median : 5.50  
 Mean   :14.50   Mean   :2002   CAMPECHE      : 16   Mean   :11.17  
 3rd Qu.:21.25   3rd Qu.:2006   CD DE MEXICO  : 16   3rd Qu.:16.30  
 Max.   :28.00   Max.   :2010   CHIHUAHUA     : 16   Max.   :41.70  
                                (Other)       :351                  
       U            CETES28            U_UN                LU         
 Min.   :0.700   Min.   : 4.300   Min.   :-1.56538   Min.   :-0.3567  
 1st Qu.:2.300   1st Qu.: 6.732   1st Qu.:-0.55131   1st Qu.: 0.8329  
 Median :3.350   Median : 8.120   Median :-0.06304   Median : 1.2088  
 Mean   :3.531   Mean   :14.319   Mean   : 0.00000   Mean   : 1.1621  
 3rd Qu.:4.500   3rd Qu.:17.500   3rd Qu.: 0.42262   3rd Qu.: 1.5041  
 Max.   :8.000   Max. 

Matrices de diagramas de dispersión 1995-2010 

In [3]:
library(car)

Loading required package: carData


In [4]:
scatterplotMatrix(~INF+U_UN+CETES28, reg.line=lm, smooth=FALSE, spread=FALSE, span=0.5, diagonal='density', data=BDatos, Periodo < 2001)

Warning message in applyDefaults(diagonal, defaults = list(method = "adaptiveDensity"), :
“unnamed diag arguments, will be ignored”Warning message in plot.window(...):
“"reg.line" is not a graphical parameter”Warning message in plot.window(...):
“"spread" is not a graphical parameter”Warning message in plot.window(...):
“"span" is not a graphical parameter”Warning message in plot.xy(xy, type, ...):
“"reg.line" is not a graphical parameter”Warning message in plot.xy(xy, type, ...):
“"spread" is not a graphical parameter”Warning message in plot.xy(xy, type, ...):
“"span" is not a graphical parameter”Warning message in title(...):
“"reg.line" is not a graphical parameter”Warning message in title(...):
“"spread" is not a graphical parameter”Warning message in title(...):
“"span" is not a graphical parameter”Warning message in plot.window(...):
“"reg.line" is not a graphical parameter”Warning message in plot.window(...):
“"spread" is not a graphical parameter”Warning message in plot.window(

En la diagonal de la matriz, se observa que la inflación y la tasa de interés presentan más de una moda y el desempleo solo una, pero es interesante como en los tres casos las modas con las mayores concentraciones se localizan en los niveles más bajos. Con respecto al diagrama de dispersión entre inflación y desempleo (segundo diagrama de la primera fila de la matriz) la relación es claramente positiva  y se identifican tres grandes concentraciones relacionadas con las modas de la inflación. En cuanto a la relación inflación y tasa de interés (tercer diagrama de la primera fila), parece que la relación es positiva y muy fuerte.

Análisis confirmatorio

En este apartado se estiman los tres tipos de modelos panel (pool, efectos fijos y aleatorios) y se aplican las pruebas de pooling y Hausman para tomar la decisión del mejor modelo en términos de consistencia. 

Estimación Pool

El comando para estimar el modelo pool incluye de derecha a izquierda, la especificación del modelo (pooling), la fuente de información que en nuestro caso se encuentra en Datos que está definida dentro de la base de datos de InflacionDesempleoCiudades.RData, la especificación de la función de inflación con respecto al diferencial del desempleo y los CETES a 28 días (INF~U_UN+CETES28), el comando que llama a la  programación de los modelos panel (plm) y la instrucción  para asignar el resultado (<-) en un objeto llamado modelo.pool.

In [6]:
library(plm)

In [7]:
modelo_pool <- plm(INF~U_UN+CETES28, data = BDatos, model = "pooling")

In [8]:
summary(modelo_pool)

Pooling Model

Call:
plm(formula = INF ~ U_UN + CETES28, data = BDatos, model = "pooling")

Balanced Panel: n = 28, T = 16, N = 448

Residuals:
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-9.53355 -2.50068 -0.95807  1.53992 16.13904 

Coefficients:
            Estimate Std. Error t-value  Pr(>|t|)    
(Intercept) 0.876198   0.356340  2.4589   0.01432 *  
U_UN        1.445834   0.319420  4.5264  7.71e-06 ***
CETES28     0.718582   0.019378 37.0820 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    46897
Residual Sum of Squares: 9964.2
R-Squared:      0.78753
Adj. R-Squared: 0.78657
F-statistic: 824.704 on 2 and 445 DF, p-value: < 2.22e-16

Estimación con Efectos Fijos 
 
Para estimar el modelo de efectos fijos se utiliza la misma línea de comandos que en el caso anterior pero con dos modificaciones: 1) En la especificación del tipo de modelo, se cambia “pooling” por “within”; y,  2)  la estimación del modelo se asignan a un nuevo objetivo llamado modelo.ef. De la misma manera, para ver los resultados se utiliza el comando summary. 

In [10]:
modelo_ef <- plm(INF ~ U_UN + CETES28, data = BDatos, model = "within")

In [11]:
summary(modelo_ef)

Oneway (individual) effect Within Model

Call:
plm(formula = INF ~ U_UN + CETES28, data = BDatos, model = "within")

Balanced Panel: n = 28, T = 16, N = 448

Residuals:
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-9.74182 -2.56814 -0.95866  1.48501 16.04426 

Coefficients:
        Estimate Std. Error t-value  Pr(>|t|)    
U_UN    1.445834   0.328766  4.3978 1.389e-05 ***
CETES28 0.718582   0.019945 36.0279 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    46848
Residual Sum of Squares: 9915.4
R-Squared:      0.78835
Adj. R-Squared: 0.77367
F-statistic: 778.484 on 2 and 418 DF, p-value: < 2.22e-16

Modelo de Efectos Aleatorios 

Finalmente para estimar el modelo de efectos aleatorios, se especifica el modelo con “random” y el mecanismo para estimar la varianza, que en este caso se utiliza el método de amemiya. Como en los casos anteriores se utiliza el comando summary, para ver los resultados de la estimación.

In [12]:
modelo_ea <- plm(INF ~ U_UN + CETES28, data = BDatos, model = "random", random.method = "amemiya")

In [13]:
summary(modelo_ea)

Oneway (individual) effect Random Effect Model 
   (Amemiya's transformation)

Call:
plm(formula = INF ~ U_UN + CETES28, data = BDatos, model = "random", 
    random.method = "amemiya")

Balanced Panel: n = 28, T = 16, N = 448

Effects:
                 var std.dev share
idiosyncratic 23.608   4.859     1
individual     0.000   0.000     0
theta: 0

Residuals:
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-9.53355 -2.50068 -0.95807  1.53992 16.13904 

Coefficients:
            Estimate Std. Error z-value  Pr(>|z|)    
(Intercept) 0.876198   0.356340  2.4589   0.01394 *  
U_UN        1.445834   0.319420  4.5264 5.999e-06 ***
CETES28     0.718582   0.019378 37.0820 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    46897
Residual Sum of Squares: 9964.2
R-Squared:      0.78753
Adj. R-Squared: 0.78657
Chisq: 1649.41 on 2 DF, p-value: < 2.22e-16

In [14]:
pooltest(modelo_pool, modelo_ef)


	F statistic

data:  INF ~ U_UN + CETES28
F = 0.076309, df1 = 27, df2 = 418, p-value = 1
alternative hypothesis: unstability


Los resultados muestran que el mejor modelo es de tipo pool, por lo que se puede concluir que es consistente con respecto al  modelo de efectos fijos. Modelo de efectos aleatorios (ea) versus el modelo de efectos fijos (ef) En sentido estricto, como el modelo pool fue mejor que el de efectos fijos no es necesario hacer la prueba de Hausman para elegir entre el modelo de efectos fijos y  aleatorios. Sin embargo, para fines ilustrativos se muestran los comandos y los resultados de la prueba de  Hausman. 

In [15]:
phtest(modelo_ea, modelo_ef)


	Hausman Test

data:  INF ~ U_UN + CETES28
chisq = 6.0896e-28, df = 2, p-value = 1
alternative hypothesis: one model is inconsistent


Aunque la prueba de Hausman  indica que el mejor modelo es el de efectos aleatorios, no tiene sentido debido a que el modelo pool es el adecuado. 

Consideraciones finales sobre los resultados

Con el objetivo de probar en el nivel más general la hipótesis sobre desempleo e inflación, en este trabajo se estimaron modelos panel con el paquete  plm del software R en sus tres especificaciones  alternativas (pool,  efectos fijos o aleatorios),    Con las pruebas pooling y de Hausman se encontró que el modelo pool es consistente con respeto a los modelos de efectos fijos y aleatorio y de acuerdo a los resultados econométricos se concluyó que tanto la tasa de desempleo bajo el mecanismo NAIRU como la tasa de interés,   tienen un efecto positivo y homogéneo sobre el proceso inflacionario de las ciudades  para el periodo 1995-2010.